In [ ]:
!pip install yfinance pandas numpy scikit-learn matplotlib tensorflow

In [ ]:
!pip install yfinance

In [ ]:
# mount my Google Drive on the VM
from google.colab import drive
drive.mount('/gdrive')

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
import yfinance as yf
import datetime


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
#Stock data gathering
start = datetime.datetime(1990, 1, 1)
end = datetime.datetime(2019,3,1)
df = yf.download("TSLA", start=start, end=end)
df=df.drop(columns=["Adj Close","Volume"])
df.reset_index(inplace=True)
df['Date'] = pd.to_datetime(df['Date'])

[*********************100%%**********************]  1 of 1 completed


In [ ]:
#Prepossesing

target_col = 'Close'
features = df[[target_col]]
target = df[target_col]

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
#Scikit

sc_model = MLPRegressor(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)
sc_model.fit(X_train_scaled, y_train)
sc_predictions = sc_model.predict(X_test_scaled)
sc_mse = mean_squared_error(y_test, sc_predictions)
print(f'Scikit Mean Squared Error (MSE): {sc_mse}')

Scikit Mean Squared Error (MSE): 0.01206594192787225


In [ ]:
#Keras

K_model = Sequential()
K_model.add(Dense(100, activation='relu', input_shape=(X_train_scaled.shape[1],)))
K_model.add(Dense(1))  # Output layer with one neuron for regression
K_model.compile(optimizer='adam', loss='mean_squared_error')
history = K_model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=0)
k_predictions = K_model.predict(X_test_scaled).flatten()
k_mse = mean_squared_error(y_test, k_predictions)
print(f'Keras Mean Squared Error (MSE): {k_mse}')


14/14 [==============================] - 0s 2ms/step
Keras Mean Squared Error (MSE): 0.0019698453503043884


In [ ]:
#Summary

total = pd.DataFrame({'Actual Prices': y_test, 'Scikit Predicted Prices': sc_predictions ,'Keras Predicted Prices': k_predictions})
total

,Actual Prices,Scikit Predicted Prices,Keras Predicted Prices
282,1.588000,1.721156,1.650993
479,2.053333,2.051983,2.031716
1317,17.396000,17.419288,17.398821
1455,16.661333,16.689979,16.665707
507,2.026667,2.032523,2.009799
...,...,...,...
1859,20.580000,20.579589,20.576572
2039,23.211332,23.189810,23.201748
548,1.894000,1.936200,1.900758
634,2.293333,2.227132,2.228975
